In [21]:
# have stack.tif file in the same directory as this file for it to work
%matplotlib qt

import numpy as np
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt

# Load image
im = Image.open('stack.tif')

# Find no. of slices
no_frames = im.n_frames

# Create separate tif files for each slice
for i in range(no_frames):
    im.seek(i)
    im.save('page_%s.tif'%(i,))
    
# create list of individual slices
images = []
for i in range(no_frames):
    temp_im = Image.open('page_%s.tif'%(i,))
    imdata = temp_im.getdata()
    x = np.reshape(imdata, (temp_im.height, temp_im.width))
    x = np.asarray(x, dtype=np.uint8)
    images.append(x)

# convert to numpy array
images_array = np.asarray(images)
np.shape(images_array) # 15, 1024, 1024
np.shape(images_array[0][0]) # 1024
type(images_array)


# create arrays of what should be red, blue and white
combined_channels = np.zeros((5, 1024, 1024))
combined_red = np.zeros((5, 1024, 1024))
combined_blue = np.zeros((5, 1024, 1024))
combined_white = np.zeros((5, 1024, 1024))
for i in range(0, 15, 3):
    combined_channels[i//3] = (images_array[i] + images_array[i+1] + images_array[i+2])
    combined_red[i//3] = (images_array[i])
    combined_blue[i//3] = images_array[i+1]
    combined_white[i//3] = images_array[i+2]

#rearrange shape for scrolling in correct plane (z)
new_stacked = np.dstack(combined_channels)
new_red = np.dstack(combined_red)
new_blue = np.dstack(combined_blue)
new_white = np.dstack(combined_white)   
    
# Fixing random state for reproducibility
np.random.seed(19680801)

# Allow scrolling through slices


class IndexTracker:
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def on_scroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        self.ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()


fig, ax = plt.subplots(1, 1)

# reshape array to be able to scroll through images
print("shape of array = ", np.shape(images_array))

print("shape of new array = ", np.shape(stacked))


tracker = IndexTracker(ax, new_stacked)


fig.canvas.mpl_connect('scroll_event', tracker.on_scroll)
plt.show()

shape of array =  (15, 1024, 1024)
shape of new array =  (1024, 1024, 15)
down -1
down -17
up 1
